In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import ELU, Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras import regularizers, losses
from tensorflow.keras import backend as K

%tensorflow_version 2.x
import tensorflow as tf

import os
import datetime
import random

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Import data from Shahname data set

In [3]:
# import dataset from txt file

main_data = np.array(())

DataSetPath = '/content/gdrive/My Drive/bsc/Final/DataSet/ferdousi_norm.txt'

dataset = pd.read_csv(DataSetPath, na_values=" ", header=None)
temp = dataset.iloc[:, 0].values

main_data = np.append(main_data, temp)

In [4]:
print(main_data[:3])
print(main_data[-1])
print(main_data[-2])

['به نام خداوند جان و خرد' 'کزین برتر اندیشه برنگذرد'
 'خداوند نام و خداوند جای']
هر آنکس که دارد هش و رای و دین
که تخم سخن من پراگنده ام


In [5]:
print("The total count of masraas is " , len(main_data))

The total count of masraas is  99217


# Extract 2000 verse from Training data set of shahname

In [6]:
X = main_data[: int(len(main_data) / 4) ]

if len(X) % 2 == 1:
    X = X[:-1]

In [7]:
print("The total count of masraas is " , len(X))

The total count of masraas is  24804


In [8]:
verses = []

for i in range(len(X)):
    if i % 2 == 0:
        verses.append(X[i] + ' <middle> ' + X[i + 1])

In [9]:
print("The total number of verses is ", len(verses))

The total number of verses is  12402


In [10]:
np.random.shuffle(verses)
verses = verses[:4000]

# Extract 1000 sample from shahname test set

In [11]:
X2 = main_data[int(len(main_data) / 4) + (int(len(main_data) / 4) % 2): ]

if len(X2) % 2 == 1:
    X2 = X2[:-1]

In [12]:
print("The total count of masraas is " , len(X2))

The total count of masraas is  74412


In [13]:
verses_test = []

for i in range(len(X2)):
    if i % 2 == 0:
        verses_test.append(X2[i] + ' <middle> ' + X2[i + 1])

In [14]:
print("The total number of verses is ", len(verses_test))

The total number of verses is  37206


In [15]:
np.random.shuffle(verses_test)
verses_test = verses_test[:2000]

# import samples which was created by our model

In [16]:
# import dataset from txt file

model_data = np.array(())

DataSetPath = '/content/gdrive/My Drive/bsc/Final/VAE/imputing_batch.txt'

dataset = pd.read_csv(DataSetPath, na_values=" ", header=None)
temp = dataset.iloc[:, 0].values

model_data = np.append(model_data, temp)

In [17]:
print("The total count of masraas is " , len(model_data))

The total count of masraas is  8000


In [18]:
verses_model = model_data[:6000]

# Create input and output dataset

In [19]:
input = np.concatenate((np.array(verses) , np.array(verses_test) , verses_model))
output = np.concatenate((np.ones(len(verses)), np.ones(len(verses_test)), np.zeros(len(verses_model))))

In [20]:
# tokenize all data

tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(input)

tokenized_input = tokenizer.texts_to_sequences(input)

In [21]:
word2index = tokenizer.word_index
index2word = tokenizer.index_word

In [22]:
# find max length for padding

max_len = 0

for i in tokenized_input:
    if len(i) > max_len:
        sen = []
        for t in i :
            sen.append(index2word[t])
        print(sen)
    max_len = max(max_len, len(i)) 

print("maximum length of sentences : ", max_len)


['مگر', 'آنکه', 'باشد', 'میان', 'دو', 'تن', '<middle>', 'سه', 'تن', 'نانهانست', 'و', 'چار', 'انجمن']
['دو', 'رانش', 'چو', 'ران', 'هیونان', 'ستبر', '<middle>', 'دل', 'شیر', 'نر', 'دارد', 'و', 'زور', 'ببر']
['تو', 'دانی', 'که', 'من', 'دوستدار', 'توام', '<middle>', 'به', 'هر', 'نیک', 'و', 'بد', 'ویژه', 'یار', 'توام']
['به', 'پیش', 'تو', 'با', 'نامور', 'چار', 'گرد', '<middle>', 'چه', 'کردم', 'تو', 'دیدی', 'ز', 'من', 'دست', 'برد']
['هم', 'آن', 'را', 'به', 'نیرنگ', 'نتوان', 'گشاد', '<middle>', 'مده', 'رنج', 'و', 'گنج', 'و', 'درم', 'را', 'به', 'باد']
['نه', 'اسپ', 'و', 'نه', 'گنج', 'و', 'نه', 'تاج', 'و', 'نه', 'تخت', '<middle>', 'نه', 'شاداب', 'در', 'باغ', 'برگ', 'درخت']
['که', 'تو', 'زان', 'فزونی', 'به', 'فرهنگ', 'و', 'بخت', '<middle>', 'به', 'فر', 'و', 'نژاد', 'و', 'به', 'تاج', 'و', 'به', 'تخت']
['به', 'دیبا', 'و', 'دینار', 'و', 'در', 'و', 'درم', '<middle>', 'به', 'بوی', 'و', 'به', 'رنگ', 'و', 'به', 'هر', 'بیش', 'و', 'کم']
maximum length of sentences :  20


In [23]:
# do padding

input_seq = pad_sequences(tokenized_input, maxlen=max_len, padding='post')

In [24]:
def discreminator(WORD_NUM, word_embedding_size, LATENT_SIZE): 
    model = Sequential()
    model.add(Embedding(
        input_dim = WORD_NUM,
        output_dim = word_embedding_size,
        input_length = max_len,
    ))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(LATENT_SIZE, return_sequences=False, kernel_regularizer=regularizers.l2(0.001))))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['binary_accuracy'])
    return model

In [25]:
WORD_NUM = len(index2word) + 1
word_embedding_size = 128
LATENT_SIZE = 128

model = discreminator(WORD_NUM, word_embedding_size, LATENT_SIZE)
model.fit(input_seq, output, epochs=200, batch_size=128, shuffle=True, validation_split=0.2)

Epoch 1/200
75/75 [==============================] - 11s 144ms/step - loss: 0.8175 - binary_accuracy: 0.6219 - val_loss: 1.0118 - val_binary_accuracy: 0.0000e+00
Epoch 2/200
75/75 [==============================] - 10s 133ms/step - loss: 0.5731 - binary_accuracy: 0.7057 - val_loss: 0.9346 - val_binary_accuracy: 0.5142
Epoch 3/200
75/75 [==============================] - 10s 133ms/step - loss: 0.4553 - binary_accuracy: 0.7841 - val_loss: 0.8765 - val_binary_accuracy: 0.6637
Epoch 4/200
75/75 [==============================] - 10s 133ms/step - loss: 0.3832 - binary_accuracy: 0.8311 - val_loss: 0.8482 - val_binary_accuracy: 0.7196
Epoch 5/200
75/75 [==============================] - 10s 134ms/step - loss: 0.3305 - binary_accuracy: 0.8599 - val_loss: 0.9800 - val_binary_accuracy: 0.6433
Epoch 6/200
75/75 [==============================] - 10s 134ms/step - loss: 0.2943 - binary_accuracy: 0.8778 - val_loss: 1.1445 - val_binary_accuracy: 0.6862
Epoch 7/200
75/75 [=============================

# Predict new data

In [26]:
verses_prediction = model_data[3000:]
input = verses_prediction
output = np.zeros(len(verses_prediction))

In [27]:
tokenized_input = tokenizer.texts_to_sequences(input)

In [28]:
input_seq = pad_sequences(tokenized_input, maxlen=max_len, padding='post')

In [29]:
y_hat = model.predict(input_seq)

In [30]:
print("Test accuracy :: %f" % (1 - np.sum(y_hat) / len(y_hat)))

Test accuracy :: 0.585087
